In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import RobustScaler
from keras.models import load_model
from joblib import dump, load

2025-04-15 02:01:55.021943: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-15 02:01:55.042624: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-15 02:01:55.052141: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-15 02:01:55.069965: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


# GPU Configuration
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    try:
        for device in physical_devices:
            tf.config.experimental.set_memory_growth(device, True)
        print("GPU memory growth enabled")
    except RuntimeError as e:
        print(e)
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Force CPU usage


GPU memory growth enabled


I0000 00:00:1744660918.941260   84139 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1744660919.009698   84139 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1744660919.010200   84139 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


# current working sequence 
## Assuming model is trained till time x //
## So assume X+15 //
## Upload it //
## wait 15 min //
## get new data //
## add it to the list // 
## scale it with load->save 
## finetune it
# repeat 

# GLOBAL VARIABLES

In [3]:
split = 0.85
sequence_length = 60
epochs = 50
learning_rate = 0.0001

In [4]:
def init():
    stock_data = pd.read_csv("runningData15.csv", delimiter=",")
    close_prices = stock_data['close'].values.reshape(-1, 1) 
    scaler = load('scaler.joblib')
    test_scaled = scaler.transform(close_prices)
    dump(scaler, 'scaler.joblib')

    X_test_new = []
    y_test_new = []
    #Dummy
    y_test_new.append(test_scaled[sequence_length -1])
    len_test = len(test_scaled)
    print(test_scaled.shape)
    for i in range(len_test - sequence_length + 1 ):
        X_test_new.append(test_scaled[i:i + sequence_length].flatten())  # Flatten the sequence
        if i + sequence_length < len_test:
            y_test_new.append(test_scaled[i + sequence_length])
    X_test_new = np.array(X_test_new)
    y_test_new = np.array(y_test_new)
    y_test_new_original = scaler.inverse_transform(y_test_new.reshape(-1, 1))
    
    return X_test_new, y_test_new, y_test_new_original

In [5]:
scaler = load('scaler.joblib')

x, y, y_original = init()
x_og = scaler.inverse_transform(x[-1].reshape(-1, 1))
print(x_og)
print("====")
print(y_original[-1])

(1849, 1)
[[3225.115]
 [3228.115]
 [3234.57 ]
 [3230.785]
 [3235.905]
 [3231.485]
 [3231.145]
 [3229.57 ]
 [3231.305]
 [3228.355]
 [3228.025]
 [3219.38 ]
 [3224.71 ]
 [3222.275]
 [3227.045]
 [3230.185]
 [3224.55 ]
 [3223.08 ]
 [3223.955]
 [3220.545]
 [3209.53 ]
 [3208.185]
 [3215.415]
 [3223.11 ]
 [3223.11 ]
 [3225.105]
 [3223.44 ]
 [3226.44 ]
 [3211.915]
 [3206.84 ]
 [3206.85 ]
 [3196.56 ]
 [3205.005]
 [3208.   ]
 [3204.675]
 [3202.53 ]
 [3204.59 ]
 [3208.605]
 [3211.25 ]
 [3209.505]
 [3205.86 ]
 [3198.19 ]
 [3198.045]
 [3202.63 ]
 [3204.94 ]
 [3206.49 ]
 [3207.65 ]
 [3210.565]
 [3210.64 ]
 [3216.22 ]
 [3214.135]
 [3213.375]
 [3210.345]
 [3204.495]
 [3208.155]
 [3210.04 ]
 [3211.36 ]
 [3209.845]
 [3210.23 ]
 [3211.225]]
====
[3211.225]


# Step 1 Predicting and uploading 

In [6]:
import requests
import numpy as np
from datetime import datetime
from uuid import UUID
from typing import Optional
from pydantic import BaseModel

class PredictionRequest(BaseModel):
    stock_id: Optional[UUID] = None
    symbol: Optional[str] = None
    opening_price: Optional[float] = None
    closing_price: Optional[float] = None
    high_price: Optional[float] = None
    low_price: Optional[float] = None
    volume: Optional[float] = None
    date: Optional[datetime] = None
    prediction_direction: Optional[bool] = None

In [7]:
def direction(y_original, y_pred):
    if y_original < y_pred:
        return True
    else:
        return False

In [14]:
def upload_prediction(y_test_new_original, results):
    API_URL = "http://127.0.0.1:8000/api/v1/prediction"
    HEADERS = {
        "Content-Type": "application/json",
        "Authorization": "Bearer your_api_key_here"
    }
    
    # Safely extract the last result value
    last_result = results[-1]
    if isinstance(last_result, np.ndarray):
        last_result = float(last_result.item())  # Properly extract scalar from array
    
    # Create the prediction request with Pydantic v2 syntax
    prediction_request = PredictionRequest(
        stock_id='ade11ce0-a353-427d-9ae7-26b948454eab',
        symbol="XAUUSD",
        opening_price=0.0,
        closing_price=last_result,
        high_price=0.0,
        low_price=0.0,
        volume=0.0,
        date=datetime.now(),
        prediction_direction=direction(y_test_new_original[-1], last_result)
    )
    
    # Convert to dict with proper serialization (Pydantic v2 syntax)
    request_data = prediction_request.model_dump()
    
    # Ensure proper serialization of special types
    if request_data['stock_id']:
        request_data['stock_id'] = str(request_data['stock_id'])
    if request_data['date']:
        request_data['date'] = request_data['date'].isoformat()
    
    try:
        response = requests.post(
            API_URL,
            json=request_data,
            headers=HEADERS,
            timeout=10
        )
        
        if response.status_code == 200:
            print("Prediction uploaded successfully")
            return True
        else:
            print(f"Failed to upload prediction: {response.status_code} - {response.text}")
            return False
    except Exception as e:
        print(f"Error uploading prediction: {str(e)}")
        return False

In [9]:
# results is like result =[], python list

def assume_and_upload(model, X_test, y_test, scaler, y_test_original, results):
    current_input = X_test[-1].reshape(1, sequence_length)
    prediction = model.predict(current_input, verbose=0)
    results.append(scaler.inverse_transform(prediction.reshape(-1, 1)))

    print(f"Predicted: {results[-1]}")

    upload_prediction(y_test_original, results)
    return results



# get New Data


In [10]:
def getLiveData(symbol: str, exchange: str, interval: str, n_bars: int):


        from tvDatafeed import TvDatafeedLive, Interval
        import pandas as pd


        username = 'your_actual_username'
        password = 'your_actual_password'

        tvl = TvDatafeedLive()

        xauusd_data = tvl.get_hist(symbol, exchange, interval=Interval.in_15_minute, n_bars=n_bars, fut_contract=None, extended_session=False, timeout=-1)
        # print(xauusd_data)

        # seis = tvl.new_seis('ETHUSDT', 'BINANCE', Interval.in_1_hour)
        # seis2 = tvl.new_seis('ETHUSDT', 'BINANCE', Interval.in_2_hour)

        # ethusdt_data = seis.get_hist(n_bars=10, timeout=-1)
        df = pd.DataFrame(xauusd_data)

        # Save to CSV
        df.to_csv('lastData.csv', index=True)

        # print("Data saved to lastData.csv")
        return df


In [11]:
import time
def get_new_data():
    while True:
        try:
            # Get new data
            getLiveData('XAUUSD', 'FOREXCOM', '15MIN', 1)
            
            # Read data - ensure no index is read
            main_df = pd.read_csv('runningData15.csv')
            new_df = pd.read_csv('lastData.csv')
            
            # Convert datetime columns
            main_df['datetime'] = pd.to_datetime(main_df['datetime'])
            new_df['datetime'] = pd.to_datetime(new_df['datetime'])
            
            # Sort by datetime to ensure chronological order
            main_df = main_df.sort_values('datetime')
            new_df = new_df.sort_values('datetime')
            
            # Get timestamps
            last_datetime = main_df['datetime'].iloc[-1]
            new_datetime = new_df['datetime'].iloc[0]
            
            if new_datetime > last_datetime:
                # Verify and clean new data before appending
                new_df = new_df[new_df['datetime'] > last_datetime]
                
                # Combine data
                combined_df = pd.concat([main_df, new_df], ignore_index=True)
                
                # Save without index and with exact column order
                combined_df.to_csv(
                    'runningData15.csv',
                    index=False,
                    columns=['datetime', 'symbol', 'open', 'high', 'low', 'close', 'volume'],
                    date_format='%Y-%m-%d %H:%M:%S'
                )
                print(f"✅ Successfully appended {len(new_df)} new records")
                print(f"Last timestamp now: {new_df['datetime'].iloc[-1]}")
                break
                
            elif new_datetime == last_datetime:
                # print("🔄 Data already exists. No update needed.")
                time.sleep(60)
                continue
                
            else:
                # print(f"⏳ New data ({new_datetime}) is older than last record ({last_datetime})")
                time.sleep(60)
                continue
                
        except pd.errors.EmptyDataError:
            print("⚠️ No data found in files. Retrying...")
            time.sleep(60)
            
        except Exception as e:
            print(f"❌ Error: {str(e)}")
            time.sleep(60)

In [44]:
get_new_data()

                              symbol      open     high       low    close  \
datetime                                                                     
2025-03-28 02:45:00  FOREXCOM:XAUUSD  3056.475  3057.13  3056.155  3056.62   

                     volume  
datetime                     
2025-03-28 02:45:00     0.0  
Data saved to lastData.csv
🔄 Data already exists. No update needed.


KeyboardInterrupt: 

# FIINTUNE

In [12]:
def finetune(model, X_test, y_test, scaler, y_test_original, results):
    second_last = X_test[-2].reshape(1, sequence_length)
    actual = y_test[-1].reshape(1, 1)
    start_time = time.time()
    model.fit(second_last,
            actual,
            epochs=20,
            batch_size = 4048,
            verbose=0)
    model.save('fine_tune_model_march_27_2nd_trial.h5')
    end_time = time.time()
    print(f"Model finetuned in {end_time - start_time} seconds")
    return results


In [13]:
model = load_model('fine_tune_model_march_27_2nd_trial.h5')
model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate * 0.1)  # Lower learning rate for fine-tuning
)
x_test, y_test, y_test_original = init()
results = []
results = finetune(model, x_test, y_test, scaler, y_test_original, results)

(831, 1)


NameError: name 'scaler' is not defined

# Server

In [16]:
model = load_model('fine_tune_model_april_15_1st_trial.h5')
model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate * 0.1)  # Lower learning rate for fine-tuning
)
while(True):
    
    X_test_new, y_test_new, y_test_new_original = init()
    results = []
    results = assume_and_upload(model, X_test_new, y_test_new, scaler, y_test_new_original, results)
    # assume_and_upload(model, X_test_new, y_test_new, scaler, y_test_new_original, results)
    results = finetune(model, X_test_new, y_test_new, scaler, y_test_new_original, results)
    current_timestamp = time.time()
    current_time = time.localtime(current_timestamp)
    current_minute = current_time.tm_min  # Extract minute (0-59)
    remainder = 15 - (current_minute % 15)
    time.sleep((remainder - 1) * 60)
    get_new_data()
    

(1849, 1)
Predicted: [[3210.373]]
Failed to upload prediction: 201 - {"message":"Prediction created successfully"}


Model finetuned in 2.797517776489258 seconds


✅ Successfully appended 1 new records
Last timestamp now: 2025-04-15 02:15:00
(1850, 1)
Predicted: [[3210.5854]]
Failed to upload prediction: 201 - {"message":"Prediction created successfully"}


Model finetuned in 0.6685924530029297 seconds


✅ Successfully appended 1 new records
Last timestamp now: 2025-04-15 02:30:00
(1851, 1)
Predicted: [[3209.77]]
Failed to upload prediction: 201 - {"message":"Prediction created successfully"}


Model finetuned in 0.6712629795074463 seconds


KeyboardInterrupt: 

In [ ]:
model = load_model('fine_tune_model_march_27.h5')
model.compile(
        loss=tf.keras.losses.MeanSquaredError(),
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate * 0.1)  # Lower learning rate for fine-tuning
    )



In [11]:
results = assume_and_upload(model, x, y, scaler, y_original, [])
print(results)

Predicted: [[3024.6055]]
Failed to upload prediction: 201 - {"message":"Prediction created successfully"}
[array([[3024.6055]], dtype=float32)]
